# Fake News Detector

Building a system to identify unreliable news articles.

In [2]:
# Making necessary imports
import pandas as pd
import matplotlib.pyplot as plt
import itertools

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from string import punctuation

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import PassiveAggressiveClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn import svm

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

## Reading and Exploring Data

In [3]:
# Read the data
df = pd.read_csv('../tensorflow-nlp_based_fake_news_detector/data/train.csv')

In [4]:
# Rows and columns in the data
df.shape

(20800, 5)

In [5]:
# Take a glimpse of the data
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
# Get more information about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
# Check for missing data in each feature/column
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# Drop unrelated features first, then drop missing data
df = df.drop(columns=['title', 'author']).dropna()

In [9]:
# Double check missing data
df.isna().sum()

id       0
text     0
label    0
dtype: int64

In [10]:
# Get the labels. 1: unreliable, 0: reliable
labels = df.label
labels.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [11]:
# Removing Null
nulls = df.isnull().sum()
nulls[nulls > 0]
df    = df.fillna(0)

# Defining X and y
X = df['text'].values.astype('U')
y = df['label'].values.astype('U')
print(X.shape) 
print(y.shape)

(20761,)
(20761,)


## Model Building

In [12]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [13]:
# Initialize a TfidfVectorizer
tfidf_vectorizer  = TfidfVectorizer(sublinear_tf=True, encoding='ISO-8859-1')

# Fit and transform train set, transform test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test  = tfidf_vectorizer.transform(X_test)

In [14]:
pipe = Pipeline(steps = [('clf', PassiveAggressiveClassifier())])
                         
                         
search_space = [{'clf': [PassiveAggressiveClassifier()]},
                {'clf': [MultinomialNB()]},
                {'clf': [BernoulliNB()]},
                {'clf': [RidgeClassifier()]},
                {'clf': [SGDClassifier()]},
                {'clf': [Perceptron()]},
                {'clf': [RandomForestClassifier()]}]
                         
gridsearch = GridSearchCV(estimator  = pipe,
                          param_grid = search_space,
                          scoring    = 'accuracy')
                         
best_model = gridsearch.fit(tfidf_train, y_train)

In [15]:
print('Best accuracy: %f using %s'%(best_model.best_score_, best_model.best_params_))

Best accuracy: 0.972383 using {'clf': SGDClassifier()}
